### 🧩 요약

- 4bit 양자화로 경량화한 모델들의 zeroshot 성능을 비교한다.
- 300개의 랜덤 train 데이터를 추론하고, 정확도(정답률)을 구한다.


### ✅ 결과 비교분석표

| 순위 | model |  accuracy (%) |  추론시간  |
| --- | --- | --- | --- |
| 1 | Qwen2-VL-7B-Instruct (4bit) | 89.000000 | 총 308.67s(샘플당 1.029s) |
| 2 | Qwen2.5-VL-3B-Instruct (4bit) | 82.333333 | 총 211.26s(샘플당 0.704s) |
| 3 | Salesforce/instructblip-flan-t5-xl | 24.66666 | ? |

- Qwen 모델 2개의 전체 데이터셋 캐글 점수랑 거의 비슷하게 나와서, 결과 신뢰도 높음. 
---

- BLIP 계열은 객관식에 약하다.
- Qwen 계열에서 모델이 커질수록 성능이 향상되는 것 같다.



---
## ⚙️ 실행 환경
- GPU: Colab (T4)  

---


## 💬 메모 / 관찰
- 입력: 이미지 + 질문 + (a,b,c,d) 선지  
- 출력: 텍스트 생성 → `extract_choice()`로 **a/b/c/d** 한 글자만 추출  
- 프롬프트에 “정답은 반드시 a,b,c,d 중 하나로만 출력” **명시**  
- 메모리 절약(양자화): **4bit quantization (bitsandbytes)** 사용  



In [1]:
# ✅ 라이브러리 설치 (4bit 양자화에 필요한 bitsandbytes 포함)
!pip -q install "transformers>=4.44.2" "accelerate>=0.34.2" "bitsandbytes>=0.43.1" --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 8.1 MB/s eta 0:00:00


### [셀 0] 드라이브 마운트 + data.zip 압축 해제 + GPU 체크

In [1]:
# ✅ 공통: 드라이브 마운트 + data.zip 해제 + GPU 상태 확인 (T4)
import os, time, subprocess, zipfile, torch

t0 = time.perf_counter()

# 💾 Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
DRIVE_DIR = "/content/drive/MyDrive/aichallenge"
ZIP_PATH  = f"{DRIVE_DIR}/data.zip"
EXTRACT_DIR = "/content"  # /content/train, /content/test 형태로 풀림

assert os.path.exists(ZIP_PATH), f"data.zip이 없습니다: {ZIP_PATH}"

# data.zip 안의 train/ 및 csv가 없으면 해제 (이미 풀려있으면 생략)
need_unzip = not (os.path.exists("/content/train") and os.path.exists("/content/train.csv"))
if need_unzip:
    print("📦 data.zip 압축 해제 중...")
    # 빠르고 안전하게: 기존 파일 덮어쓰기 방지 위해 먼저 정리(선택)
    # !rm -rf /content/train /content/test /content/*.csv
    !unzip -q "{ZIP_PATH}" -d "{EXTRACT_DIR}"
else:
    print("📦 이미 압축 해제됨: /content/train, /content/test, /content/*.csv 존재")

# ⚙️ GPU 체크
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
print("GPU:", gpu_name)
!nvidia-smi

# ⏱️ 시간 요약
dt = time.perf_counter() - t0
print(f"[공통 준비] 총 {dt:.2f}s")


Mounted at /content/drive
📦 data.zip 압축 해제 중...
GPU: Tesla T4
Thu Oct 23 17:56:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |          

### [셀 1] 공통 유틸 + pilot_300.csv 로드

In [2]:
# ✅ 공통 유틸 + 파일럿 데이터 로드
import pandas as pd, torch, time, random
from PIL import Image

PILOT_CSV = "/content/drive/MyDrive/aichallenge/pilot_300.csv"
assert os.path.exists(PILOT_CSV), f"pilot_300.csv이 없습니다: {PILOT_CSV}"

# 파일럿 데이터 고정 로드
pilot_df = pd.read_csv(PILOT_CSV)
assert {"id","path","question","a","b","c","d","answer"}.issubset(pilot_df.columns)
print("파일럿 샘플 수:", len(pilot_df))

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

def time_summary(t0, n, title=""):
    dt = time.perf_counter() - t0
    print(f"[{title}] 총 {dt:.2f}s | 샘플당 {dt/max(1,n):.3f}s")

def extract_choice(text: str) -> str:
    t = str(text).strip().lower()
    lines = [l.strip() for l in t.splitlines() if l.strip()]
    if not lines: return "a"
    last = lines[-1]
    if last in ["a","b","c","d"]: return last
    for tok in last.split():
        if tok in ["a","b","c","d"]: return tok
    return "a"

def true_label(v: str) -> str:
    v = str(v).strip().lower()
    return v[0] if v and v[0] in ["a","b","c","d"] else "a"

# 공통 저장 경로
RESULT_DIR = "/content/drive/MyDrive/aichallenge/results"
os.makedirs(RESULT_DIR, exist_ok=True)
print("결과 저장 폴더:", RESULT_DIR)

# 경로가 /content 기준이므로, 이미지 파일도 /content/train/... 를 가리켜야 함
# (pilot_300.csv가 data.zip에서 뽑은 경로라면 그대로 OK)


파일럿 샘플 수: 300
device: cuda
결과 저장 폴더: /content/drive/MyDrive/aichallenge/results


---
## ▶ 모델별 실행: 아래 네 개 중, 한 번에 하나만 실행

- 실행 후 결과가 Drive에 저장되면,
다음 모델로 갈 때 [셀 0→1→2]만 다시 실행하고, 아래 다른 모델 셀을 실행하세요.

- 각 셀 마지막에 del model, processor; torch.cuda.empty_cache()로 최대한 비웁니다 (T4는 완전 비움이 아니므로 재시작 권장).





---
## [셀 10] Qwen 2.5-VL-3B-Instruct (4bit, 한국어 프롬프트)

In [3]:
import time, torch, os
from tqdm import tqdm
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig

MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"  # 필요 시 "Qwen/Qwen2-VL-3B-Instruct"
print("Loading:", MODEL_ID)

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID, quantization_config=bnb, device_map="auto", trust_remote_code=True
)

SYSTEM_INSTRUCT = (
    "You are a helpful visual question answering assistant. "
    "Answer using exactly one letter among a, b, c, or d. No explanation."
)
def build_mc_prompt(q, a, b, c, d):
    return (f"{q}\n(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
            "정답은 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요.\n정답:")

preds = []
t0 = time.perf_counter()
for i in tqdm(range(len(pilot_df)), desc="Qwen2.5-VL-3B Pilot", unit="sample"):
    row = pilot_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")
    messages = [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
        {"role": "user", "content": [
            {"type": "image", "image": img},
            {"type": "text",  "text": build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[img], return_tensors="pt").to(device)
    with torch.no_grad():
        out_ids = model.generate(
            **inputs, max_new_tokens=2, do_sample=False,
            eos_token_id=processor.tokenizer.eos_token_id,
        )
    out_text = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
    preds.append(extract_choice(out_text))

time_summary(t0, len(pilot_df), "Qwen2.5-VL-3B 파일럿")
golds = [true_label(x) for x in pilot_df["answer"].tolist()]
acc = (sum(1 for g,p in zip(golds, preds) if g==p) / len(golds)) * 100
print(f"✅ [Qwen2.5-VL-3B] 정확도: {acc:.2f}%")

# 저장
out_path = os.path.join(RESULT_DIR, "preds_qwen2p5_vl_3b_pilot300.csv")
pilot_df.assign(pred=preds)[["id","answer","pred"]].to_csv(out_path, index=False)
print("저장:", out_path)

del model, processor
torch.cuda.empty_cache()


Loading: Qwen/Qwen2.5-VL-3B-Instruct


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

Qwen2.5-VL-3B Pilot: 100%|██████████| 300/300 [03:31<00:00,  1.42sample/s]

[Qwen2.5-VL-3B 파일럿] 총 211.26s | 샘플당 0.704s
✅ [Qwen2.5-VL-3B] 정확도: 82.33%
저장: /content/drive/MyDrive/aichallenge/results/preds_qwen2p5_vl_3b_pilot300.csv


### ✅ [InstructBLIP] 정확도: 24.67%
- 정답 추출 함수(파서) 문제 => 파서 함수 교체


In [7]:
# 🔧 기존 extract_choice 대신 이걸로 교체
import re
def extract_choice(text: str) -> str:
    """
    모델 출력에서 a/b/c/d 하나만 견고하게 추출:
    - 대소문자 허용
    - 'c.' 'd)' '[a]' 같은 문장부호 허용
    - 'Answer: c' 같은 패턴 우선 인식
    실패하면 기본 'a'
    """
    t = (str(text) or "").lower()

    # 1) "answer: c" 같은 명시 패턴
    m = re.search(r'answer\s*[:\-]?\s*([abcd])\b', t)
    if m: return m.group(1)

    # 2) 토큰 경계의 단독 문자
    m = re.search(r'\b([abcd])\b', t)
    if m: return m.group(1)

    # 3) 뒤에 문장부호가 붙은 경우: c.  c)  c,  c]
    m = re.search(r'\b([abcd])[\)\]\}\>\.\,\!\?:;]\b?', t)
    if m: return m.group(1)

    # 4) 줄 끝에 글자가 오고 개행되는 경우 등
    m = re.search(r'([abcd])\s*$', t)
    if m: return m.group(1)

    return "a"




---
### [셀 11] InstructBLIP (Salesforce/instructblip-flan-t5-xl, 4bit)

In [10]:
import time, torch, os
from tqdm import tqdm
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration, BitsAndBytesConfig

MODEL_ID = "Salesforce/instructblip-flan-t5-xl"
print("Loading:", MODEL_ID)

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

processor = InstructBlipProcessor.from_pretrained(MODEL_ID)
model = InstructBlipForConditionalGeneration.from_pretrained(
    MODEL_ID, quantization_config=bnb, device_map="auto"
)

# SYSTEM_INSTRUCT_EN = (
#     "You are a helpful visual question answering assistant. "
#     "Answer using exactly one letter among a, b, c, or d. No explanation."
# )
# def build_mc_prompt_en(q, a, b, c, d):
#     return (f"{SYSTEM_INSTRUCT_EN}\n\nQuestion: {q}\n"
#             f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
#             "Answer (a/b/c/d only):")

# preds = []
# t0 = time.perf_counter()
# for i in tqdm(range(len(pilot_df)), desc="InstructBLIP Pilot", unit="sample"):
#     row = pilot_df.iloc[i]
#     img = Image.open(row["path"]).convert("RGB")
#     prompt = build_mc_prompt_en(row["question"], row["a"], row["b"], row["c"], row["d"])
#     inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
#     # 예측 직전에 출력 텍스트를 일부 로그로 확인
#     if i < 5:
#         print(f"[raw {i}] {out_text!r}")
#     with torch.no_grad():
#         out_ids = model.generate(**inputs, max_new_tokens=2, do_sample=False)
#     out_text = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
#     preds.append(extract_choice(out_text))

# time_summary(t0, len(pilot_df), "InstructBLIP 파일럿")
# golds = [true_label(x) for x in pilot_df["answer"].tolist()]
# acc = (sum(1 for g,p in zip(golds, preds) if g==p) / len(golds)) * 100
# print(f"✅ [InstructBLIP] 정확도: {acc:.2f}%")

# out_path = os.path.join(RESULT_DIR, "preds_instructblip_pilot300.csv")
# pilot_df.assign(pred=preds)[["id","answer","pred"]].to_csv(out_path, index=False)
# print("저장:", out_path)

# del model, processor
# torch.cuda.empty_cache()


Loading: Salesforce/instructblip-flan-t5-xl


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 계속 a 하나로 쏠림 현상.. 정확도 25% 쯤 나옴..
InstructBLIP에서는 generate()를 그대로 쓰되 “a/b/c/d 중 하나만 출력하게 강제”하는 게 훨씬 효과적이야.
즉 “편향된 자유생성”이 아니라 “forced decoding”으로 정답 후보만 남겨두는 방식으로 제어해야 해.

In [13]:
import torch, time, os
from tqdm import tqdm
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration, BitsAndBytesConfig

MODEL_ID = "Salesforce/instructblip-flan-t5-xl"
bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
processor = InstructBlipProcessor.from_pretrained(MODEL_ID)
model = InstructBlipForConditionalGeneration.from_pretrained(
    MODEL_ID, quantization_config=bnb, device_map="auto"
).eval()

# SYSTEM_INSTRUCT_EN = (
#     "You are a helpful visual question answering assistant. "
#     "Answer only with one letter: a, b, c, or d."
# )
SYSTEM_INSTRUCT_EN = (
    "You are a helpful visual question answering assistant. "
    "정답은 다음 중 하나입니다: a, b, c, d."
)
def build_mc_prompt(q, a, b, c, d):
    return (f"{SYSTEM_INSTRUCT_EN}\n\n"
            f"Question: {q}\n"
            f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
            "The correct answer is:")

letters = ["a","b","c","d"]

# ⏱️ 평가 루프
preds = []
t0 = time.perf_counter()
for i in tqdm(range(len(pilot_df)), desc="InstructBLIP Pilot", unit="sample"):
    row = pilot_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")
    prompt = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])
    inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=2,
            do_sample=False,
            temperature=0.0,
            top_p=0.1,
        )

    text_out = processor.batch_decode(out, skip_special_tokens=True)[0].lower().strip()

    # 🔍 추가 정규화
    import re
    m = re.search(r"\b([abcd])\b", text_out)
    pred = m.group(1) if m else "a"
    if i < 5:
        print(f"[raw {i}] {text_out!r} -> {pred}")
    preds.append(pred)

time_summary(t0, len(pilot_df), "InstructBLIP 파일럿 (Forced decode)")
golds = [true_label(x) for x in pilot_df["answer"].tolist()]
acc = (sum(1 for g,p in zip(golds, preds) if g==p) / len(golds)) * 100
print(f"✅ [InstructBLIP Forced] 정확도: {acc:.2f}%")

out_path = os.path.join(RESULT_DIR, "preds_instructblip_pilot300.csv")
pilot_df.assign(pred=preds)[["id","answer","pred"]].to_csv(out_path, index=False)
print("저장:", out_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

InstructBLIP Pilot:   0%|          | 1/300 [00:00<03:08,  1.59sample/s]

[raw 0] 'a' -> a


InstructBLIP Pilot:   1%|          | 2/300 [00:01<02:37,  1.89sample/s]

[raw 1] 'a' -> a


...

[raw 3] 'a' -> a


InstructBLIP Pilot:   2%|▏         | 5/300 [00:02<01:57,  2.51sample/s]

[raw 4] 'a' -> a


InstructBLIP Pilot:  18%|█▊        | 53/300 [00:20<01:37,  2.54sample/s]


KeyboardInterrupt: 

---
### [셀 12] BLIP2 (Salesforce/blip2-flan-t5-xl, 4bit)

In [4]:
import time, torch, os
from tqdm import tqdm
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BitsAndBytesConfig

MODEL_ID = "Salesforce/blip2-flan-t5-xl"
print("Loading:", MODEL_ID)

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

processor = Blip2Processor.from_pretrained(MODEL_ID)
model = Blip2ForConditionalGeneration.from_pretrained(
    MODEL_ID, quantization_config=bnb, device_map="auto"
)

SYSTEM_INSTRUCT_EN = (
    "You are a helpful visual question answering assistant. "
    "Answer using exactly one letter among a, b, c, or d. No explanation."
)
def build_mc_prompt_en(q, a, b, c, d):
    return (f"{SYSTEM_INSTRUCT_EN}\n\nQuestion: {q}\n"
            f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
            "Answer (a/b/c/d only):")

preds = []
t0 = time.perf_counter()
for i in tqdm(range(len(pilot_df)), desc="BLIP2 Pilot", unit="sample"):
    row = pilot_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")
    prompt = build_mc_prompt_en(row["question"], row["a"], row["b"], row["c"], row["d"])
    inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out_ids = model.generate(**inputs, max_new_tokens=3, do_sample=False)
    out_text = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
    preds.append(extract_choice(out_text))

time_summary(t0, len(pilot_df), "BLIP2 파일럿")
golds = [true_label(x) for x in pilot_df["answer"].tolist()]
acc = (sum(1 for g,p in zip(golds, preds) if g==p) / len(golds)) * 100
print(f"✅ [BLIP2] 정확도: {acc:.2f}%")

out_path = os.path.join(RESULT_DIR, "preds_blip2_pilot300.csv")
pilot_df.assign(pred=preds)[["id","answer","pred"]].to_csv(out_path, index=False)
print("저장:", out_path)

del model, processor
torch.cuda.empty_cache()


Loading: Salesforce/blip2-flan-t5-xl


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

KeyboardInterrupt: 

---
### [셀 13] Qwen2-VL-7B-Instruct (4bit, 큰 모델)

In [3]:
import time, torch, os
from tqdm import tqdm
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig

MODEL_ID = "Qwen/Qwen2-VL-7B-Instruct"  # 필요 시 "Qwen/Qwen2.5-VL-7B-Instruct"
print("Loading:", MODEL_ID)

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID, quantization_config=bnb, device_map="auto", trust_remote_code=True
)

SYSTEM_INSTRUCT = (
    "You are a helpful visual question answering assistant. "
    "Answer using exactly one letter among a, b, c, or d. No explanation."
)
def build_mc_prompt(q, a, b, c, d):
    return (f"{q}\n(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
            "정답은 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요.\n정답:")

preds = []
t0 = time.perf_counter()
for i in tqdm(range(len(pilot_df)), desc="Qwen2-VL-7B Pilot", unit="sample"):
    row = pilot_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")
    messages = [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
        {"role": "user", "content": [
            {"type": "image", "image": img},
            {"type": "text",  "text": build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[img], return_tensors="pt").to(device)
    with torch.no_grad():
        out_ids = model.generate(
            **inputs, max_new_tokens=2, do_sample=False,
            eos_token_id=processor.tokenizer.eos_token_id,
        )
    out_text = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
    preds.append(extract_choice(out_text))

time_summary(t0, len(pilot_df), "Qwen2-VL-7B 파일럿")
golds = [true_label(x) for x in pilot_df["answer"].tolist()]
acc = (sum(1 for g,p in zip(golds, preds) if g==p) / len(golds)) * 100
print(f"✅ [Qwen2-VL-7B] 정확도: {acc:.2f}%")

out_path = os.path.join(RESULT_DIR, "preds_qwen2_vl_7b_pilot300.csv")
pilot_df.assign(pred=preds)[["id","answer","pred"]].to_csv(out_path, index=False)
print("저장:", out_path)

del model, processor
torch.cuda.empty_cache()


Loading: Qwen/Qwen2-VL-7B-Instruct


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

Qwen2-VL-7B Pilot: 100%|██████████| 300/300 [05:08<00:00,  1.03s/sample]


[Qwen2-VL-7B 파일럿] 총 308.67s | 샘플당 1.029s
✅ [Qwen2-VL-7B] 정확도: 89.00%
저장: /content/drive/MyDrive/aichallenge/results/preds_qwen2_vl_7b_pilot300.csv


---
### [셀 99] (선택) 저장된 결과들 한 번에 비교

네 모델을 모두 돌린 뒤, 마지막에 한 번만 실행해 비교표를 봐줘.

In [5]:
import glob, pandas as pd, os

PILOT_CSV = "/content/drive/MyDrive/aichallenge/pilot_300.csv"
RESULT_DIR = "/content/drive/MyDrive/aichallenge/results"

pilot = pd.read_csv(PILOT_CSV)[["id","answer"]].copy()
files = sorted(glob.glob(os.path.join(RESULT_DIR, "preds_*_pilot300.csv")))
print("발견된 결과 파일:", [os.path.basename(f) for f in files])

out = pilot.copy()
for f in files:
    df = pd.read_csv(f)
    col = os.path.basename(f).replace("preds_","").replace("_pilot300.csv","")
    # 🔧 answer 컬럼 제거 후 id/pred만 남김
    if "answer" in df.columns:
        df = df.drop(columns=["answer"])
    out = out.merge(df.rename(columns={"pred": f"pred_{col}"}), on="id", how="left")

def acc(pred_col):
    return (out["answer"].str[0].str.lower() == out[pred_col].str[0].str.lower()).mean()*100

summary = []
for c in out.columns:
    if c.startswith("pred_"):
        summary.append((c, acc(c)))
summary_df = pd.DataFrame(summary, columns=["model","accuracy(%)"]).sort_values("accuracy(%)", ascending=False)
print("📊 모델별 정확도(파일럿 300 동일 샘플):")
display(summary_df)

out.head(10)


발견된 결과 파일: ['preds_instructblip_pilot300.csv', 'preds_qwen2_vl_7b_pilot300.csv', 'preds_qwen2p5_vl_3b_pilot300.csv']
📊 모델별 정확도(파일럿 300 동일 샘플):


,model,accuracy(%)
1,pred_qwen2_vl_7b,89.000000
2,pred_qwen2p5_vl_3b,82.333333
0,pred_instructblip,24.666667


,id,answer,pred_instructblip,pred_qwen2_vl_7b,pred_qwen2p5_vl_3b
0,train_2256,d,a,d,d
1,train_2694,a,a,a,a
2,train_1305,a,a,a,a
3,train_3818,b,a,b,b
4,train_3102,b,a,b,b
5,train_1750,c,a,c,c
6,train_3185,d,a,d,d
7,train_1075,b,a,b,b
8,train_3537,b,a,b,b
9,train_3371,d,a,d,d
